In [1]:
import numpy as np
np.set_printoptions(precision=5,suppress=True)

In [2]:
#adjacency dictionary
#right,up,left,down
neighbours = {0:[0,1,0,0],
              1:[1,2,3,0],
              2:[2,2,2,1],
              3:[4,3,1,3],
              4:[4,5,3,4],
              5:[5,5,5,4]}

states = len(neighbours)
#right,up,left,down
direction_probability = [0.2,0.5,0.1,0.2]
stationary_distribution = {0:7,
                           1:5,
                           2:4,
                           3:1,
                           4:5,
                           5:7}

if(len(stationary_distribution) != states):
    print("Stationary distribution doesnt match number of states!")

In [3]:
#Metropolis
metropolis_mat = np.zeros((states,states))

for state,adjacent in zip(neighbours.keys(),neighbours.values()):
    accept_probability = 0.0 #the probability to move to any of the neighbouring states
    for direction in range(4):
        if(adjacent[direction]!=state):
            acceptance_ratio = min(1,stationary_distribution[adjacent[direction]]/stationary_distribution[state])
            transition_probability = (1/4)*acceptance_ratio
            accept_probability += transition_probability
            metropolis_mat[state,adjacent[direction]] = transition_probability
    metropolis_mat[state,state] = 1-accept_probability    
    
print("Metropolis matrix\n",metropolis_mat)

Metropolis matrix
 [[0.82143 0.17857 0.      0.      0.      0.     ]
 [0.25    0.5     0.2     0.05    0.      0.     ]
 [0.      0.25    0.75    0.      0.      0.     ]
 [0.      0.25    0.      0.5     0.25    0.     ]
 [0.      0.      0.      0.05    0.7     0.25   ]
 [0.      0.      0.      0.      0.17857 0.82143]]


In [4]:
#Metropolis-Hastings
MH_mat = np.zeros((states,states))
proposal_mat = np.zeros((states,states))

#could be inserted in above algorithm for speedup
for state,adjacent in zip(neighbours.keys(),neighbours.values()):
    for direction in range(4):
        proposal_mat[state,adjacent[direction]] += direction_probability[direction]
        
for state,adjacent in zip(neighbours.keys(),neighbours.values()):
    accept_probability = 0.0 #the probability to move to any of the neighbouring states
    for direction in range(4):
        if(adjacent[direction]!=state):
            acceptance_ratio = min(1, \
                (stationary_distribution[adjacent[direction]]*proposal_mat[adjacent[direction],state])/ \
                (stationary_distribution[state]*proposal_mat[state,adjacent[direction]]))
            transition_probability = direction_probability[direction]*acceptance_ratio
            accept_probability += transition_probability
            MH_mat[state,adjacent[direction]] = transition_probability
    MH_mat[state,state] = 1-accept_probability
    
for num,line in enumerate(np.sum(MH_mat,axis=1)):
    if(abs(line-1)>0.0000001):
        print(f"Sum of line {num} is not zero")
        break
if(MH_mat.min()<0):
    print("Minimum value is negative")
    
print("Metropolis Hastings matrix\n",MH_mat)

Metropolis Hastings matrix
 [[0.85714 0.14286 0.      0.      0.      0.     ]
 [0.2     0.62    0.16    0.02    0.      0.     ]
 [0.      0.2     0.8     0.      0.      0.     ]
 [0.      0.1     0.      0.7     0.2     0.     ]
 [0.      0.      0.      0.04    0.68    0.28   ]
 [0.      0.      0.      0.      0.2     0.8    ]]


In [5]:
def rejectionRate(mat):
    rejectionRates = []
    for i in range(mat.shape[0]):
        rejectionRates.append(mat[i,i])
    return rejectionRates

print("Metropolis rejection rates:\n",rejectionRate(metropolis_mat))
print("Metropolis Hastings rejection rates:\n",rejectionRate(MH_mat))

Metropolis rejection rates:
 [0.8214285714285714, 0.5, 0.75, 0.5, 0.7, 0.8214285714285714]
Metropolis Hastings rejection rates:
 [0.8571428571428572, 0.62, 0.8, 0.7, 0.6799999999999999, 0.8]


In [6]:
eigenvalues, eigenvectors = np.linalg.eig(MH_mat.transpose())
for i in range(states):
    if(abs(eigenvalues[i]-1)<0.0000000001):
        print("eigenvalue:", eigenvalues[i])
        print(eigenvectors[:, i] / sum(eigenvectors[:, i]))

eigenvalue: 0.9999999999999989
[0.24138 0.17241 0.13793 0.03448 0.17241 0.24138]


In [7]:
eigenvalues, eigenvectors = np.linalg.eig(metropolis_mat.transpose())
for i in range(states):
    if(abs(eigenvalues[i]-1)<0.0000000001):
        print("eigenvalue:", eigenvalues[i])
        print(eigenvectors[:, i] / sum(eigenvectors[:, i]))

eigenvalue: 0.999999999999999
[0.24138 0.17241 0.13793 0.03448 0.17241 0.24138]


In [8]:
#sanity check
z = sum(stationary_distribution.values())
print([x/z for x in stationary_distribution.values()])

[0.2413793103448276, 0.1724137931034483, 0.13793103448275862, 0.034482758620689655, 0.1724137931034483, 0.2413793103448276]
